# Multi-label classification -- top-push loss

In [1]:
%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.optimize import check_grad
from scipy.misc import logsumexp

from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sys.path.append('src')
from evaluate import avgPrecision, avgPrecisionK, evaluationPrecision, avgF1, evaluationF1
from datasets import create_dataset, dataset_names, nLabels_dict

In [3]:
dataset_names

['yeast', 'scene', 'bibtex', 'bookmarks', 'delicious', 'mediamill']

In [4]:
data_ix = 2

In [5]:
dataset_name = dataset_names[data_ix]
nLabels = nLabels_dict[dataset_name]
print(dataset_name, nLabels)

bibtex 159


In [6]:
data_dir = 'data'
fmodel = os.path.join(data_dir, 'tp-' + dataset_name + '.pkl')
fperf  = os.path.join(data_dir, 'perf-tp.pkl')

Load dataset.

In [7]:
X_train, Y_train = create_dataset(dataset_name, train_data=True)
X_test,  Y_test  = create_dataset(dataset_name, train_data=False)

In [8]:
print('%-30s %s' % ('Dataset:', dataset_name))
print('%-30s %d' % ('Number of training examples:', X_train.shape[0]))
print('%-30s %d' % ('Number of test examples:', X_test.shape[0]))
print('%-30s %d' % ('Number of labels:', Y_train.shape[1]))
print('%-30s %d' % ('Number of features:', X_train.shape[1]))

Dataset:                       bibtex
Number of training examples:   4880
Number of test examples:       2515
Number of labels:              159
Number of features:            1836


The sigmoid function.

In [9]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

## top-push loss

Multi-label learning with top push loss.

In [10]:
#n = 100000
#m = 8000
#randM = np.random.rand(n, m)

In [11]:
#%%timeit
#print(np.sum(randM))

In [12]:
#%%timeit
#print(np.dot(np.ones(n), np.dot(randM, np.ones(m))))  # more efficient than np.sum()

In [13]:
def obj_toppush(w, X, Y, C, r=1):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, C = 1 / lambda
            - r: parameter for log-sum-exp approximation
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(r > 0)
    assert(C > 0)
    
    W = w.reshape(K, D)  # theta
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    # instead of using diagonal matrix to scale each row of a matrix with a different factor,
    # we use Mat * Vec[:, None] which is more memory efficient
    
    KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    A_diag = 1.0 / KPosAll
    AY = Y * A_diag[:, None]
    
    T1 = np.dot(X, W.T)  # N by K
    #m0 = np.max(T1)  # underflow in np.exp(r*T1 - m1)
    m0 = 0.5 * (np.max(T1) + np.min(T1))
    m1 = r * m0
    print('----------------')
    print(m0, np.min(T1))
    
    T2 = np.multiply(1 - Y, np.exp(r * T1 - m1))  # N by K
    B_tilde_diag = np.dot(T2, np.ones(K))
    print(np.max(B_tilde_diag), np.min(B_tilde_diag))  # big numbers here, can cause overflow in T3
    
    #T3 = np.exp(-T1 + m0) * np.power(B_tilde_diag, 1.0 / r)[:, None]
    #T4 = np.multiply(AY, np.log1p(T3))
    T3 = (-T1 + m0) + np.log(B_tilde_diag)[:, None]
    print(np.min(T3), np.max(T3))
    m2 = 0.5 * (np.min(T3) + np.max(T3))
    #T4 = np.logaddexp(0, T3)
    T4 = np.logaddexp(-m2, T3-m2) + m2
    T5 = np.multiply(AY, T4)  
    
    J = np.dot(w, w) * 0.5 / C + np.dot(np.ones(N), np.dot(T5, np.ones(K))) / N
    
    #T5 = 1.0 / (1.0 + np.divide(1.0, T3))
    #T5 = np.divide(T3, 1 + T3)
    T6 = np.exp(T3 - T4)
    O_diag = np.dot(np.multiply(Y, T6), np.ones(K))
    T7 = A_diag * (1.0 / B_tilde_diag) * O_diag
    
    G1 = np.dot(np.multiply(AY, T6).T, -X)
    G2 = np.dot((T2 * T7[:, None]).T, X)
    
    G = W / C + (G1 + G2) / N
    
    return (J, G.ravel())

In [14]:
def obj_toppush_loop(w, X, Y, C, r=1):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, C = 1 / lambda
            - r: parameter for log-sum-exp approximation
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(r > 0)
    assert(C > 0)
    
    W = w.reshape(K, D)  # theta
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    
    for n in range(N):
        for k in range(K):
            if Y[n, k] == 1:
                s1 = np.sum([np.exp(r * np.dot(W[j, :] - W[k, :], X[n, :])) for j in range(K) if Y[n, j] == 0])
                J += np.log1p(np.power(s1, 1.0 / r)) / KPosAll[n]
    J = np.dot(w, w) * 0.5 / C + J / N
    
    for k in range(K):
        for n in range(N):
            if Y[n, k] == 1:
                t1 = np.sum([np.exp(r * np.dot(W[j, :] - W[k, :], X[n, :])) for j in range(K) if Y[n, j] == 0])
                t2 = -1.0 / (1 + np.power(t1, -1.0 / r))
                G[k, :] = G[k, :] + X[n, :] * t2 / KPosAll[n]
            else:
                sk = 0.0
                for k1 in range(K):
                    if Y[n, k1] == 1:
                        t3 = np.sum([np.exp(r * np.dot(W[j,:] - W[k1, :], X[n, :])) \
                                     for j in range(K) if Y[n, j] == 0])
                        t4 = np.exp(r * np.dot(W[k, :] - W[k1, :], X[n, :]))
                        sk += t4 / (np.power(t3, 1.0 - 1.0 / r) + t3)
                G[k, :] = G[k, :] + X[n, :] * sk / KPosAll[n]
                        
    G = W / C + G / N
    
    return (J, G.ravel())

Check gradient

In [15]:
#aa = np.array([0,1,2, 0])
#print(aa)
#print([aa[i] for i in range(4) if aa[i] != 0])
#print([aa[i] if aa[i] != 0 else 10 for i in range(4)])

In [16]:
#X_train = X_train[:10, :]
#Y_train = Y_train[:10, :]

In [17]:
C = 1
#w0 = np.random.rand(Y_train.shape[1] * X_train.shape[1]) - 0.5
w0 = 0.001 * np.random.randn(Y_train.shape[1] * X_train.shape[1])

In [18]:
%%script false
eps = 1.49e-08
w = np.zeros_like(w0)
for i in range(len(w0)):
    wi1 = w0.copy()
    wi2 = w0.copy()
    wi1[i] = wi1[i] - eps
    wi2[i] = wi2[i] + eps
    J1, _ = obj_toppush_loop(wi1, X_train, Y_train, C)
    J2, _ = obj_toppush_loop(wi2, X_train, Y_train, C)
    w[i] = (J2 - J1) / (2 * eps)
    #print(w[i])
J, w1 = obj_toppush_loop(w0, X_train, Y_train, C)
diff = w1 - w
np.sqrt(np.dot(diff, diff))

In [19]:
%%script false
check_grad(lambda w: obj_toppush_loop(w, X_train, Y_train, C)[0], 
           lambda w: obj_toppush_loop(w, X_train, Y_train, C)[1], w0)

In [20]:
%%script false
check_grad(lambda w: obj_toppush(w, X_train, Y_train, C)[0], 
           lambda w: obj_toppush(w, X_train, Y_train, C)[1], w0)

In [21]:
%%script false
print('%15s %15s %15s %15s' % ('J_Diff', 'J_loop', 'J_vec', 'G_Diff'))
for e in range(-6, 10):
    C = 10**(e)
    #w0 = init_var(X_train, Y_train)
    J,  G  = obj_toppush_loop(w0, X_train, Y_train, C)
    J1, G1 = obj_toppush(w0, X_train, Y_train, C)
    Gdiff = G1 - G
    #print('%-15g %-15g %-15g' % (J1 - J, J, J1))
    print('%15g %15g %15g %15g' % (J1 - J, J, J1, np.dot(Gdiff, Gdiff)))

In [22]:
class MLC_toppush(BaseEstimator):
    """All methods are necessary for a scikit-learn estimator"""
    
    def __init__(self, C=1, r=1):
        """Initialisation"""
        
        assert C > 0
        assert r > 0
        self.C = C
        self.r = r
        self.trained = False
        
    def fit(self, X_train, Y_train):
        """Model fitting by optimising the objective"""
        opt_method = 'L-BFGS-B' #'BFGS' #'Newton-CG'
        options = {'disp': 1, 'maxiter': 10**5, 'maxfun': 10**5} # , 'iprint': 99}
        print('\nC: %g, r: %g' % (self.C, self.r))
            
        N, D = X_train.shape
        K = Y_train.shape[1]
        #w0 = np.random.rand(K * D) - 0.5  # initial guess in range [-1, 1]
        w0 = 0.001 * np.random.randn(K * D)
        opt = minimize(obj_toppush, w0, args=(X_train, Y_train, self.C, self.r), \
                       method=opt_method, jac=True, options=options)
        if opt.success is True:
            self.W = np.reshape(opt.x, (K, D))
            self.trained = True
        else:
            sys.stderr.write('Optimisation failed')
            print(opt.items())
            self.trained = False
            
            
    def decision_function(self, X_test):
        """Make predictions (score is real number)"""
        
        assert self.trained is True, "Can't make prediction before training"
        D = X_test.shape[1]
        return np.dot(X_test, self.W.T)
        
    
    def predict(self, X_test):
        """Make predictions (score is boolean)"""
        
        preds = sigmoid(self.decision_function(X_test))
        #return (preds >= 0)
        assert self.TH is not None
        return preds >= self.TH
    
    
    def score(self, X, Y):
        """Compute scoring metric"""
        return self.score_F1(X, Y)
        #return self.score_precisionK(X, Y)
        
    
    def score_precisionK(self, X, Y):
        """Compute Precision@K"""
        allPreds = self.decision_function(X)
        return avgPrecisionK(Y, allPreds)
    
    
    def score_F1(self, X, Y):
        """Compute F1"""
        #allPreds = self.predict(X)
        #return avgF1(Y, allPreds)
        allPreds = sigmoid(self.decision_function(X))
        ranges = np.arange(0.01, 1, 0.01)
        F1 = [f1_score(Y, allPreds >= th, average='samples') for th in ranges]
        bestix = np.argmax(F1)
        self.TH = ranges[bestix]
        return F1[bestix]
        
    # inherit from BaseEstimator instead of re-implement
    #
    #def get_params(self, deep = True):
    #def set_params(self, **params):

In [23]:
old_settings = np.seterr(all='ignore')  # seterr to known value
np.seterr(all='raise')
#np.seterr(all='ignore')
#np.seterr(**old_settings)  # restore settings

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'ignore', 'under': 'ignore'}

In [24]:
%%script false
#%mprun
clf = MLC_toppush(C=10000)
clf.fit(X_train, Y_train)
#pkl.dump(clf, open(fmodel, 'wb'))
#%memit model.fit(X_train[:30], Y_train[:30])
#%mprun -f minimize model.fit(X_train[:100], Y_train[:100])
#%mprun -f _minimize_slsqp model.fit(X_train[:10], Y_train[:10])

In [25]:
ranges = range(-6, 7)
#ranges = [x for x in ranges if x not in range(-6, -3)]
parameters = [{'C': sorted([10**(e) for e in ranges] + [3 * 10**(e) for e in ranges])}]

if not os.path.exists(fmodel):
    clf = GridSearchCV(MLC_toppush(), parameters, cv=5, n_jobs=1)
    clf.fit(X_train, Y_train)
    pkl.dump(clf, open(fmodel, 'wb'))
else:
    clf = pkl.load(open(fmodel, 'rb'))

if hasattr(clf, 'best_params_'):
    print("\nBest parameters set found on development set:")
    print(clf.best_params_)


C: 1e-06, r: 1
----------------
-0.00084070106348 -0.0490973966332
158.477977281 131.047379984
4.8542565851 5.10796088818
----------------
0.000714368077395 -0.0402930453534
158.20389652 130.965894832
4.85891160537 5.10354664046
----------------
1.24424383448e-06 -1.67375758091e-07
157.999803466 130.999836759
4.87519644311 5.06259518924


/home/dawei/apps/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)



C: 1e-06, r: 1
----------------
-0.00633123194613 -0.0647509337401
159.584172519 132.022706978
4.85606733168 5.10723305079
----------------
0.00541040749539 -0.0445132089322
157.38277375 130.136826679
4.86257708577 5.10609315104
----------------
9.567893001e-07 -1.69227537185e-07
157.999848895 130.999874429
4.87519653511 5.06259518644


/home/dawei/apps/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)



C: 1e-06, r: 1
----------------
-0.00278387653857 -0.0557797760633
158.758582467 131.247951459
4.85555923717 5.1107879587
----------------
0.00237978972854 -0.0429247080883
157.923463167 130.79427878
4.85507149964 5.09827766093
----------------
1.14277560473e-06 -1.68619912383e-07
157.999819508 130.999850017
4.87519650228 5.0625951877


...

----------------
-16.5183708297 -358.579271837
2.08466542369e+40 2643124426.87
-286.93847441 401.038045288
----------------
-17.680412448 -373.705901051
5.61808151611e+41 8760568508.9
-297.980592233 416.996678552
----------------
-17.1976580844 -379.924252594
5.05115294934e+41 5534014400.44
-304.464535112 423.486944412
----------------
-16.9333815643 -376.812379382
1.30662857113e+41 4243409257.23
-302.250156201 419.77826129
----------------
-16.0728858733 -362.964842064
1.93361682591e+39 1755399026.4
-291.69442505 404.271583993
----------------
-15.3655750221 -353.855795824
1.21667263987e+38 854910414.26
-285.009712536 394.500700603
----------------
-14.0562064005 -334.735840633
4.27803458567e+35 229311366.824
-270.585213107 374.04020743
----------------
-14.0358589689 -334.197478591
3.83087230778e+35 227476587.257
-270.172876228 373.074011476
----------------
-14.4217081593 -337.575230122
1.11105578891e+36 340546601.017
-272.429402733 376.344446916
----------------
-14.8025417146 -339

----------------
-19.1868120413 -235.596666887
2.77725367867e+30 44057952354.0
-172.519260303 263.378217966
----------------
-18.6986683484 -229.406176714
4.66987092644e+29 27122505558.7
-168.053672487 256.992891831
----------------
-18.963504144 -232.764743901
1.22856071671e+30 35288529746.8
-170.485561107 260.452073405
----------------
-18.7147638949 -230.047992088
4.52298693012e+29 27556376604.7
-168.610942054 257.618355569
----------------
-18.627367007 -229.534400363
3.66957933605e+29 25255508705.7
-168.340600035 257.003809041
----------------
-18.3247120795 -227.608204094
2.41057595689e+29 18677284331.4
-167.215384235 254.704653076
----------------
-17.9679845921 -225.119232472
1.95089223179e+29 13084153345.2
-165.664072237 251.817138609
----------------
-17.9764870537 -225.468407453
2.57296830273e+29 13188939016.2
-165.959631353 252.158585456
----------------
-17.9034928315 -225.112540429
3.17229180314e+29 12251088071.4
-165.799163775 251.723410051
----------------
-17.793949252

----------------
-10.4747628012 -151.943219205
8.15091349154e+22 6606693.24079
-112.022418271 171.922104835
----------------
-10.1554041954 -149.065313809
4.29407231273e+22 4793132.1289
-109.936194073 168.762717774
----------------
-9.82454635829 -145.835127803
2.40275566163e+22 3437389.18658
-107.377014422 165.189424425
----------------
-9.58104321014 -143.460592861
1.74802371029e+22 2691698.5674
-105.474100904 162.548206632
----------------
-9.69812171809 -144.602290659
2.0369335879e+22 3027493.437
-106.389720747 163.815042051
----------------
-9.59707170029 -143.768001682
1.87752630384e+22 2735392.24484
-105.808255915 162.875946037
----------------
-9.51038905255 -143.086954551
1.81182341018e+22 2507783.68811
-105.331199399 162.109474684
----------------
-9.44083018819 -143.012253785
1.94749185707e+22 2339284.78477
-105.546133034 161.986683198
----------------
-9.3249424567 -142.63279407
1.88003204799e+22 2083227.20338
-105.498091718 161.544403664
----------------
-9.22230720024 -14

----------------
-0.452943436225 -90.3003972397
7.81266942526e+14 280.16521512
-73.9666423472 105.954486622
----------------
-0.26956753824 -89.275071384
4.4523959717e+14 233.039159089
-73.2488004869 104.856021544
----------------
-0.0921798936428 -88.2712988521
2.67722399025e+14 195.028667581
-72.4808883299 103.766330644
----------------
0.214803868472 -86.7248214037
2.20641599169e+14 143.291284116
-71.6130707195 102.132950015
----------------
0.0547509325276 -87.531112441
2.44042409961e+14 168.274777893
-72.0664066984 102.983905225
----------------
0.117472627095 -87.2546045938
2.20438590683e+14 158.046683298
-71.862642388 102.680528227
----------------
0.49019935108 -85.7665678898
1.90100940365e+14 108.850092703
-71.1487532406 101.111803072
----------------
0.640877043599 -85.3416778603
2.0525906558e+14 93.6825476097
-71.0043736733 100.662376794
----------------
0.588527023917 -85.6547622198
2.52849270385e+14 98.7302450292
-71.3199203504 101.001835346
----------------
0.627330510717

----------------
4.62490493299 -10.5180620388
544.711505109 1.51489504713
-12.1689052953 18.3984252714
----------------
4.62984527292 -10.5092538108
540.26671463 1.50741511895
-12.1724751283 18.3864489652
----------------
4.6302468458 -10.5072718757
538.244997611 1.50679483086
-12.1738092423 18.3811748789
----------------
4.63521597721 -10.4977851874
530.792342754 1.49928474015
-12.180033535 18.3674277208
----------------
4.63455760122 -10.5131177249
524.920941536 1.50022852179
-12.2044781624 18.3663419011
----------------
4.63498664617 -10.5031259454
528.726205358 1.49961332559
-12.1885760432 18.3667902705
----------------
4.63263922046 -10.5069703419
528.845102157 1.50312472308
-12.1879695179 18.3700938552
----------------
4.6321976646 -10.5082049558
528.444808709 1.50377885835
-12.1881792076 18.3719183658
----------------
4.63093268769 -10.5125205366
528.640961281 1.50567083784
-12.1880560517 18.3751161771
----------------
4.62950719763 -10.5161021896
528.651836816 1.50780654565
-12

In [26]:
if hasattr(clf, 'cv_results_'):
    for mean, std, params in zip(clf.cv_results_['mean_test_score'], \
                                 clf.cv_results_['std_test_score'], \
                                 clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.058 (+/-0.002) for {'C': 1e-06}
0.058 (+/-0.002) for {'C': 3e-06}
0.058 (+/-0.002) for {'C': 1e-05}
0.058 (+/-0.002) for {'C': 3.0000000000000004e-05}
0.058 (+/-0.002) for {'C': 0.0001}
0.058 (+/-0.002) for {'C': 0.00030000000000000003}
0.058 (+/-0.002) for {'C': 0.001}
0.058 (+/-0.002) for {'C': 0.003}
0.058 (+/-0.002) for {'C': 0.01}
0.067 (+/-0.004) for {'C': 0.03}
0.106 (+/-0.011) for {'C': 0.1}
0.113 (+/-0.014) for {'C': 0.30000000000000004}
0.116 (+/-0.014) for {'C': 1}
0.121 (+/-0.013) for {'C': 3}
0.146 (+/-0.015) for {'C': 10}
0.222 (+/-0.024) for {'C': 30}
0.321 (+/-0.016) for {'C': 100}
0.376 (+/-0.016) for {'C': 300}
0.397 (+/-0.018) for {'C': 1000}
0.401 (+/-0.016) for {'C': 3000}
0.393 (+/-0.021) for {'C': 10000}
0.355 (+/-0.018) for {'C': 30000}
0.302 (+/-0.013) for {'C': 100000}
0.257 (+/-0.012) for {'C': 300000}
0.213 (+/-0.010) for {'C': 1000000}
0.169 (+/-0.014) for {'C': 3000000}


In [27]:
preds_train = clf.decision_function(X_train)
preds_test  = clf.decision_function(X_test)

In [28]:
print('Training set:')
perf_dict_train = evaluationPrecision(Y_train, preds_train)
print()
print('Test set:')
perf_dict_test = evaluationPrecision(Y_test, preds_test)

Training set:
Average Precision@3: 0.6430, 0.004
Average Precision@5: 0.4420, 0.004
Average Precision@K: 0.9649, 0.002

Test set:
Average Precision@3: 0.3931, 0.006
Average Precision@5: 0.2883, 0.004
Average Precision@K: 0.5044, 0.008


In [ ]:
preds_train_bin = clf.predict(X_train)
preds_test_bin  = clf.predict(X_test)

In [34]:
model = MLC_toppush(C=3000)
model.fit(X_train, Y_train)
model.score(X_train, Y_train)


C: 3000, r: 1
----------------
-0.00252638894581 -0.0583789712618
158.827109337 131.47040699
4.85433096304 5.11685205712
----------------
2.55880815598 -0.39131053199
17.0361840028 10.6010365163
-0.329295114846 5.76308758563
----------------
6.49270140478 -1.38280106537
16.2594177363 0.22236765993
-7.85300113822 10.4747682949
----------------
5.16732455539 -1.52545982753
19.6611074284 0.833514075456
-6.15370926861 9.04713538294
----------------
5.38514455053 -1.91150132296
23.7946373033 0.67026350625
-6.90937640848 9.68685343851
----------------
6.26783365486 -2.82557823748
80.4306012267 0.276291240774
-9.15614548839 12.3318591181
----------------
5.39739681751 -9.09304958996
4590.07725346 0.655196955037
-12.9829371882 16.5542021488
----------------
0.780104295928 -25.2021984238
56427764.696 67.4516910834
-19.7081178946 33.8406033852
----------------
-4.73967767823 -43.7536758473
1.0841309009e+12 17785.513871
-28.3147568867 53.367627787
----------------
-7.42955824782 -54.6791551914
3

----------------
4.6360123921 -10.5043764051
527.666837512 1.49804110227
-12.1881554058 18.3706307903
----------------
4.63621620628 -10.5032879688
527.570449697 1.49773755453
-12.1882849616 18.3690567345
----------------
4.63623314108 -10.5021869835
527.853211356 1.49771391782
-12.1875111328 18.368355268
----------------
4.6370102855 -10.4988820231
528.158674883 1.49655691875
-12.1884376297 18.3664312159
----------------
4.63667193137 -10.4991951634
528.44944478 1.49706216274
-12.1864905561 18.3668301375


/home/dawei/apps/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.79790554352514464

In [35]:
preds_train_bin = model.predict(X_train)
preds_test_bin  = model.predict(X_test)

In [37]:
#f1_train = evaluationF1(Y_train, preds_train_bin)
f1_train = f1_score(Y_train, preds_train_bin, average='samples')
#f1_test = evaluationF1(Y_test, preds_test_bin)
f1_test = f1_score(Y_test, preds_test_bin, average='samples')
print('Training set:', f1_train)
print()
print('Test set:', f1_test)

Training set: 0.797905543525

Test set: 0.402333830925


/home/dawei/apps/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [ ]:
perf_dict_train.update(f1_train)
perf_dict_test.update(f1_test)

In [ ]:
print(perf_dict_train)
print(perf_dict_test)

In [ ]:
perf_dict = {'Train': perf_dict_train, 'Test': perf_dict_test}
if os.path.exists(fperf):
    _dict = pkl.load(open(fperf, 'rb'))
    if dataset_name not in _dict:
        _dict[dataset_name] = perf_dict
else:
    _dict = {dataset_name: perf_dict}
pkl.dump(_dict, open(fperf, 'wb'))

In [ ]:
pkl.load(open(fperf, 'rb'))

## Result analysis

Plot the histogram of the $K$-th highest prediction scores, where $K$ is the number of positive labels for a given example.

In [ ]:
K_train = Y_train.sum(axis=1).astype(np.int)
K_test = Y_test.sum(axis=1).astype(np.int)

In [ ]:
K_pred_train = [sorted(preds_train[n, :])[::-1][K_train[n]-1] for n in range(X_train.shape[0])]

In [ ]:
K_pred_test = [sorted(preds_test[n, :])[::-1][K_test[n]-1] for n in range(X_test.shape[0])]

In [ ]:
fig = plt.figure(figsize=[10, 3])
plot_data = [K_pred_train, K_pred_test]
plt.suptitle('Histogram on %s dataset' % dataset_name)
for i in range(2):
    ax = plt.subplot(1,2,i+1)
    ax.hist(plot_data[i], bins=20)
    ax.set_title('%s set' % ('Training' if i == 0 else 'Test'))
    ax.set_xlabel('K-th highest prediction score')
    ax.set_ylabel('Frequency')
plt.savefig(dataset_name + '_kscore_hist.svg')

In [ ]:
#print(K_pred_train[0], K_train[0])

In [ ]:
#print(K_pred_test[0], K_test[0])

In [ ]:
#preds_train[0, :]

In [ ]:
#preds_test[0, :]

In [ ]:
precisions_train = [avgPrecision(Y_train, preds_train, k) for k in range(1, nLabels+1)]
precisions_test  = [avgPrecision(Y_test,  preds_test,  k) for k in range(1, nLabels+1)]

In [ ]:
precisionK_train = avgPrecisionK(Y_train, preds_train)
precisionK_test  = avgPrecisionK(Y_test,  preds_test)

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(precisions_train, ls='--', c='r', label='Train')
plt.plot(precisions_test,  ls='-',  c='g', label='Test')
plt.plot([precisionK_train for k in range(nLabels)], ls='-', c='r', label='Train, Precision@K')
plt.plot([precisionK_test  for k in range(nLabels)], ls='-', c='g', label='Test, Precision@K')
plt.xticks(np.arange(nLabels), np.arange(1,nLabels+1))
plt.xlabel('k')
plt.ylabel('Precision@k')
plt.legend(loc='best')
plt.title('MLC w. Top-push Loss on ' + dataset_name + ' dataset')
plt.savefig(dataset_name + '_tp.svg')